# Sentimental Analysis for Mobile Phones Comments

Importing Essentials 

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import re
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
def np_SaveData(myList,filename):
    np.array(myList).dump(open(filename+'.npy', 'wb'))
    
def np_LoadData(filename):
    myArray = np.load(open(filename+'.npy', 'rb'))
    return myArray

def SaveDictionary(Dic,DicName):
    #This saves all the key-value pair dictionaries
    pickle_out = open(DicName,"wb")
    pickle.dump(Dic, pickle_out)
    pickle_out.close()
    
def ReadDictionary(DicName):
    pickle_in = open(DicName,"rb")
    diction = pickle.load(pickle_in)
    return diction


In [3]:
path=os.getcwd()
x=path+"\\Dictionary\\"
os.chdir(x)
print ("Entering Dictionary Directory to load Data >>")
print ("\t"+os.getcwd())
Docs=np_LoadData("Docs")
print ("\t"+"Docs Loaded")
os.chdir(path)
print ("Returning to Main Directory")
print ("\t"+os.getcwd())

Entering Dictionary Directory to load Data >>
	C:\Users\Asad\Python\CodePython\FYP\GsmScrapper\Dictionary
	Docs Loaded
Returning to Main Directory
	C:\Users\Asad\Python\CodePython\FYP\GsmScrapper


For every phone we have to calculate a Rating of the phone based on the Sentiment Analysis of Comments. So for each phone, we will calculate a score of rating out of 100% based on all comments for that phone

In [4]:
def LabeledComments(Docs):
    Comment=[]
    Sentiment=[]
    counterpos=0
    counterneg=0
    counter=0
    Files=len(Docs)
    for Phones in Docs:
        for Comments in Phones:
            analysis= TextBlob(Comments)
            if(analysis.sentiment.polarity!=0.0):
                if(analysis.sentiment.polarity>0.0):
                    #Positive
                    if(counterpos<=80000):
                        counterpos+=1
                        Comment.append(Comments)
                        Sentiment.append("Positive")
                    else:
                        pass
                else:
                    #Negative
                    Comment.append(Comments)
                    Sentiment.append("Negative")
                    counterneg+=1
        if(counterneg>80000):
            break
        print("Progress {:2.1%}".format(counter / Files), end="\r")
        counter+=1
    return Comment,Sentiment

In [5]:
Comment,Sentiment=LabeledComments(Docs)

In [6]:
cn=0
cp=0
for text in Sentiment:
    if (text=='Positive'):
        cp+=1
    else:
        cn+=1
print ("Negative to Positive Comment Ratio is: ",cp,":",cn)

Negative to Positive Comment Ratio is:  80001 : 80017


In [8]:
print (len(Comment),len(Sentiment))
data = pd.DataFrame({'Text':Comment, 'Sentiment':Sentiment})

160018 160018


In [9]:
data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Sentiment'] == 'Positive'].size)
print(data[ data['Sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 2000
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)

160002
160034


In [22]:
embed_dim = 128
lstm_out = 196
mymodel = Sequential()
mymodel.add(embeddings(max_features,embed_dim,input_length = X.shape[1], dropout=0.2))
mymodel.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
mymodel.add(Dense(2,activation='softmax'))
mymodel.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(mymodel.summary())

TypeError: 'module' object is not callable

In [ ]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")